<a href="https://colab.research.google.com/github/POOJA-BFRS01535/chatbot/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install OpenAI library
!pip install -U -q openai tenacity

from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/upgrad/GenAI_Course_Master/Chatbot_Project/')
!ls

# Import the libraries
import pandas as pd
from IPython.display import display, HTML
# Set the display width to control the output width
pd.set_option('display.width', 100)
# Read the dataset and read the Laptop Dataset



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.2 MB/s eta 0:00:00
Mounted at /content/drive
OPENAI_API_Key.txt


In [ ]:
# Import the libraries
import os, json, ast
import openai
from tenacity import retry, wait_random_exponential, stop_after_attempt

# read the OpenAI API key
openai.api_key = open("OPENAI_API_Key.txt", "r").read().strip()
os.environ['OPENAI_API_KEY'] = openai.api_key

In [20]:
INTENTS = {
    "order_status": ["order status", "track my order", "where is my order"],
    "product_info": ["tell me about", "product details", "product info"],
    "return_policy": ["return policy", "how to return", "refund policy"],
    "shipping_info": ["shipping details", "delivery time", "shipping info"],
}


In [21]:
print(INTENTS.keys())


dict_keys(['order_status', 'product_info', 'return_policy', 'shipping_info'])


In [22]:
def initialize_conversation():
    '''
    Returns a list [{"role": "system", "content": system_message}]
    '''

    delimiter = "####"

    system_message = f"""
    You are an experienced customer support agent for an ecommerce direct to customer brand and your goal is to handle customer support queries of customers.
    You will be handling 5 types of user queries. Check user order, check status of shipment, get product info, answer questions about return and refund policy.
    In case user asks questions which you do not know how to handle redirect user to manual customer support.

    You need to ask relevant questions and understand the user intent first and reply back with what is user intent out of below intents.
    If you are unable to identify intent continue the conversation till you find the intent.
    In below dictionary, keys are intents and the list of values for each key represent some sample input which user can give for that intent.
    Based on these sample inputs identify the intent of user.
    {delimiter}
    {INTENTS}
    {delimiter}

    Your output should be a single word intent out of {INTENTS.keys()}. If you are unable to identify indent then continue conversation with user.

    {delimiter}

    """
    conversation = [{"role": "system", "content": system_message}]
    return conversation



In [23]:
# Define a Chat Completions API call
# Retry up to 6 times with exponential backoff, starting at 1 second and maxing out at 20 seconds delay
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
def get_chat_completions(input, json_format = False):
    MODEL = 'gpt-3.5-turbo'

    system_message_json_output = """<<. Return output in JSON format to the key output.>>"""

    # If the output is required to be in JSON format
    if json_format == True:
        # Append the input prompt to include JSON response as specified by OpenAI
        input[0]['content'] += system_message_json_output

        # JSON return type specified
        chat_completion_json = openai.chat.completions.create(
            model = MODEL,
            messages = input,
            response_format = { "type": "json_object"},
            seed = 1121)

        output = json.loads(chat_completion_json.choices[0].message.content)

    # No JSON return type specified
    else:
        chat_completion = openai.chat.completions.create(
            model = MODEL,
            messages = input,
            seed = 1211)

        output = chat_completion.choices[0].message.content

    return output

In [24]:
system_msg = initialize_conversation()
debug_conversation = system_msg
print(system_msg[0]["content"])


    You are an experienced customer support agent for an ecommerce direct to customer brand and your goal is to handle customer support queries of customers. 
    You will be handling 5 types of user queries. Check user order, check status of shipment, get product info, answer questions about return and refund policy.
    In case user asks questions which you do not know how to handle redirect user to manual customer support.
    
    You need to ask relevant questions and understand the user intent first and reply back with what is user intent out of below intents. 
    If you are unable to identify intent continue the conversation till you find the intent. 
    In below dictionary, keys are intents and the list of values for each key represent some sample input which user can give for that intent. 
    Based on these sample inputs identify the intent of user.
    ####
    {'order_status': ['order status', 'track my order', 'where is my order'], 'product_info': ['tell me about', 'pro

In [25]:
user_input1 = "Hi"
debug_conversation.append({'role':'user','content':user_input1})

debug_response_assistant = get_chat_completions(debug_conversation) ## Chat Completions API


In [26]:
debug_conversation.append(({"role": "assistant", "content": debug_response_assistant}))


In [28]:
debug_conversation

[{'role': 'system',
  'content': "\n    You are an experienced customer support agent for an ecommerce direct to customer brand and your goal is to handle customer support queries of customers. \n    You will be handling 5 types of user queries. Check user order, check status of shipment, get product info, answer questions about return and refund policy.\n    In case user asks questions which you do not know how to handle redirect user to manual customer support.\n    \n    You need to ask relevant questions and understand the user intent first and reply back with what is user intent out of below intents. \n    If you are unable to identify intent continue the conversation till you find the intent. \n    In below dictionary, keys are intents and the list of values for each key represent some sample input which user can give for that intent. \n    Based on these sample inputs identify the intent of user.\n    ####\n    {'order_status': ['order status', 'track my order', 'where is my ord

In [29]:
user_input2 = "I want to know more about refund policy"
debug_conversation.append({'role':'user','content':user_input2})

debug_response_assistant = get_chat_completions(debug_conversation) ## Chat Completions API
print(debug_response_assistant)



return_policy


In [31]:
def check_intent_identified(assistant_response):
  if assistant_response in INTENTS.keys():
    return True

print(check_intent_identified(debug_response_assistant))


True


In [ ]:
def get_order_info():
  """
  get order id from user - get_order_id
  once obtained - pull out order info from db/shopify API and make it json. - extract_order_info
  Based on what user asked prepare response - prepare_order_response
  """
  pass

def get_product_info():
  """
  get ecommerce site url from settings for example phool.co
  first check what user has asked. Form a seach query for it. create_search_query
  Then go to website of the ecommerce company and search for the product in it. perform_search
  then formulate the results and respond back - respond_to_user_query
  """
  pass

def get_return_policy_info():
  """
  get details of return and refund policy of company from urls of the pages in their websites in settings - prepare_refund_return_policy
  answer user query based on the return and refund policy - return_return_refund_policy_info
  """
  pass

def get_shipment_info():
  """
  get shipment awb no from user -
  """
  pass


In [ ]:
INTENT_to_action_map = {
    "order_status": get_order_info,
    "product_info": get_product_info,
    "return_policy": get_return_policy_info,
    "shipping_info": get_shipment_info,
}

if (check_intent_identified):
  action = INTENT_to_action_map.get(debug_response_assistant)
  action()
else:
  debug_conversation.append(({"role": "assistant", "content": debug_response_assistant}))
